### **MODELING**

Since we've been able to properly segment different properties of documents. The next part is to build the model that will be responsible for recognising individual characters.
**Deep Learning** will be used to classify the images into 75 different groups based on their diacritics. 
The classes for this model are quite much, so we are further going to make try implementing **multi-label classification** and then compare results.

Since our total dataset contains just 41 samples for each class. This might be a negative pull down for deep learning models since they are always data hungry.

**Training data** = 38 samples for each class(**Total = *2850**)

**Testing data** = 3 samples for each class(**Total = 2850**)

In [54]:
from fastai.vision import *
from fastai.metrics import error_rate, accuracy
import warnings
warnings.filterwarnings('ignore')

In [55]:
path = Path('/home/max/Pictures/data')

In [56]:
data = ImageDataBunch.from_folder(path, train='train', ds_tfms=get_transforms(do_flip = False), valid='test', size=28, bs=64)#.normalize(imagenet_stats)

In [57]:
for itemList in ["train_dl", "valid_dl", "fix_dl", "test_dl"]:
    itemList = getattr(data, itemList)
    if itemList: itemList.x.convert_mode = "L"

In [58]:
data.one_batch()[0].size()

torch.Size([64, 1, 28, 28])

**MODEL ARCHITECTURE (RESNET34)**

In [59]:
learn = cnn_learner(data, models.resnet34, metrics = [accuracy, error_rate])

In [60]:
learn.model[0][0] = nn.Conv2d(1, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)

In [ ]:
interp = ClassificationInterpretation.from_learner(learn)

losses,idxs = interp.top_losses()

len(data.valid_ds)==len(losses)==len(idxs)

In [63]:
learn.unfreeze()

**Model still need to train for estimately 20 more epochs.. More accuracy can be attained**
**Error_rate** also tells more abou

In [ ]:
learn.fit_one_cycle(10)

epoch,train_loss,valid_loss,accuracy,error_rate,time
0,3.518884,2.901143,0.258772,0.741228,04:58
1,3.043840,2.358664,0.394737,0.605263,05:14


In [ ]:
interp = ClassificationInterpretation.from_learner(learn)

losses,idxs = interp.top_losses()

len(data.valid_ds)==len(losses)==len(idxs)

In [ ]:
#interp.plot_top_losses(9, figsize=(15,11))

In [ ]:
#interp.plot_confusion_matrix(figsize=(12,12), dpi=60)